In [2]:
#nbi:hide_in
#nbi:hide_out
!pip3 install matplotlib
!pip3 install keras
!pip3 install spectral
!pip3 install seaborn
!pip3 install scikit-image
!pip3 install tensorflow
!pip3 install sklearn

     |████████████████████████████████| 10.3 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 63.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 62.3 MB/s eta 0:00:01
     |████████████████████████████████| 636 kB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 27.4 MB 228 kB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 33.6 MB/s eta 0:00:01     |██████████████                  | 1.8 MB 33.6 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 285 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 29.2 MB 98 kB/s  eta 0:00:01
     |████████████████████████████████| 1.6 MB 81.4 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 77.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 65.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 71.7 MB/s eta 0:00:01
     |████████████████████████████████

     |████████████████████▌           | 2.4 MB 29.1 MB/s eta 0:00:01

     |████████████████████████████████| 3.8 MB 29.1 MB/s eta 0:00:01

     |████████████████████████████████| 3.8 MB 29.1 MB/s 
     |████████████████████████████████| 14.8 MB 67.0 MB/s eta 0:00:01               | 5.8 MB 67.0 MB/s eta 0:00:01�████████████▏   | 13.1 MB 67.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 14.8 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 66.4 MB/s eta 0:00:0100:01
     |████████████████████████████████| 129 kB 90.1 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 753 kB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 462 kB 56.7 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.8 MB/s  eta 0:00:01
     |████████████████████████████████| 781 kB 56.3 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 88.7 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 72.5 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 62.5 MB/s eta 0:00:01
     |████████████

In [1]:
#nbi:hide_in
%matplotlib inline
import spectral as sp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from spectrum import rank_normalizer,scale_max_min,scale_white_dark,plot_spectra,zoom
from spectrum_ml import cluster_pixels_kmeans,detect_aphid,blob_detector
from ipywidgets import interact, interactive, widgets
from IPython.display import display
import io

In [10]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['SPECTRAL_DATA'] = '/home/plavainne/Documents/BIM/'

In [11]:
wl = widgets.FileUpload(
    accept='',  # A/home/plavainne/Documents/BIM/06022020/06022020/VL1grF/ccepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description = 'Wave_Length'
)

display(wl)

FileUpload(value={}, description='Wave_Length')

In [123]:
# Create text widget for output
exp_name = widgets.Text()
current_dir = widgets.Text()

In [124]:
options1 = {
    'title': 'Exp_name',
}

options2 = {
    'title': 'dossier courant',
}

In [125]:
# Define function to bind value of the input to the output variable 
def name(x):
    exp_name.value = str(x)
    
def dossier_courant(x):
    current_dir.value = str(x)

In [126]:
en = interactive(name, x = "VL1grF", options = options1)

In [127]:
cd = interactive(dossier_courant, x = "/home/plavainne/Documents/BIM/06022020/06022020/")

In [128]:
display(en, options = options1)

interactive(children=(Text(value='VL1grF', description='x'), Output()), _dom_classes=('widget-interact',))

In [129]:
display(cd)

interactive(children=(Text(value='/home/plavainne/Documents/BIM/06022020/06022020/', description='x'), Output(…

In [135]:
button = widgets.Button(description="Rechargez les donnees")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        
        wave_lengths = pd.read_csv(io.BytesIO(wl.value[list(wl.value.keys())[0]]['content']), header=None, index_col=0)
        image_path = current_dir.value+exp_name.value+'/capture/'+exp_name.value+'.hdr'
        dark_ref_path = current_dir.value+exp_name.value+'/capture/DARKREF_'+exp_name.value+'.hdr'
        white_ref_path = current_dir.value+exp_name.value+'/capture/WHITEREF_'+exp_name.value+'.hdr'
        
        img = sp.open_image(image_path)

        dark_ref = sp.open_image(dark_ref_path)
        dark_spectrum = dark_ref.asarray().reshape(-1,dark_ref.shape[2]).mean(axis=0)

        white_ref = sp.open_image(white_ref_path)
        white_spectrum = white_ref.asarray().reshape(-1,white_ref.shape[2]).mean(axis=0)
        
        img_norm = scale_white_dark(img.asarray(),dark_spectrum=dark_spectrum,white_spectrum=white_spectrum)
        img_norm = scale_max_min(img_norm)
        
        img_norm_reshape = img_norm.reshape(-1,img_norm.shape[-1])
        
        print(img.shape)
        plot_spectra(img,  wave_lengths=(29,113,227), grid_step=20)

button.on_click(on_button_clicked)

Button(description='Rechargez les donnees', style=ButtonStyle())

Output()